# **Reocmmendation System**

**Goal:** Show reviews that are similar to the one I’m reading (same movie).

**Why embeddings (not TF-IDF)?**
- Text is in French (maybe with a bit of English). Multilingual sentence embeddings understand meaning across both languages better than TF-IDF.

**Preprocessing (make a clean table)**
- Keep only: `id` and cleaned `content` (so we can get the original review later by `id`).
- Drop missing values.
- Remove HTML.
- Remove punctuation and extra spaces.
- Lowercase text.
- Drop rows that become empty after cleaning.

**Modeling**
- Use a multilingual SentenceTransformer to turn each review into a vector (embedding).
- Use cosine similarity to find the closest reviews to the chosen one.
- Return top-k(5) similar reviews with their `id` and similarity score.

In [2]:
from google.colab import files
#Import data
uploaded = files.upload()

Saving fightclub_critiques.csv to fightclub_critiques.csv
Saving interstellar_critiques.csv to interstellar_critiques.csv


In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
fightclub = pd.read_csv('fightclub_critiques.csv')
fightclub.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",Sons of Anarchy.,199,Gand-Alf,55207
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack,191,Ano,2333
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721


In [5]:
interstellar = pd.read_csv('interstellar_critiques.csv')
interstellar.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"<p>Chers lecteurs, chères lectrices,</p>\n\n<p...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,<p>Malgré ce que j'entends dire ou lis sur le ...,Tous les chemins mènent à l'Homme,347,blig,133793
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"<p>Un grand film, pour moi, c'est un film qui ...",Rage against the dying of the light.,453,Samu-L,41951
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,<p>Tout en n'étant absolument pas un admirateu...,Inspacetion,273,Nushku,24046


#**1 . Data Preprocessing**

##**Check missing values**


In [ ]:
fightclub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       1000 non-null   int64 
 1   URL                      1000 non-null   object
 2   rating                   1000 non-null   int64 
 3   review_date_creation     1000 non-null   object
 4   review_date_last_update  1000 non-null   object
 5   review_hits              1000 non-null   int64 
 6   review_content           999 non-null    object
 7   review_title             649 non-null    object
 8   gen_review_like_count    1000 non-null   int64 
 9   username                 1000 non-null   object
 10  user_id                  1000 non-null   int64 
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


In [ ]:
interstellar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       1000 non-null   int64 
 1   URL                      1000 non-null   object
 2   rating                   1000 non-null   int64 
 3   review_date_creation     1000 non-null   object
 4   review_date_last_update  1000 non-null   object
 5   review_hits              1000 non-null   int64 
 6   review_content           1000 non-null   object
 7   review_title             832 non-null    object
 8   gen_review_like_count    1000 non-null   int64 
 9   username                 1000 non-null   object
 10  user_id                  1000 non-null   int64 
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


In [6]:
#Remove the Null value in fightclub and create new dataframes for the cleaning
f_content = fightclub[['review_content', 'id']].dropna()
i_content = interstellar[['review_content', 'id']]
f_content

,review_content,id
0,Fight Club sort à la fin 1999 sur les écrans. ...,20761
1,Tout a été déjà dit sur le film de David Finch...,30753423
2,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",12057488
3,<p>Beaucoup pensent que ce film n'est qu'un él...,10939046
4,"<p>Objet d’un véritable culte, Fight Club a ma...",1966438
...,...,...
995,"<p>Bonjour, </p><p>Regardez ce film car : oui....",297093474
996,"<p>Ce film est très surcoté, il as tout pour e...",205504617
997,<p>Banger</p>,318641521
998,"<p><a href=""https://boxd.it/5UblLD"">https://bo...",325535346


**We keep track on the content id in order to get it later after the recommandation analysis**

In [7]:
#Remove Html tags from our content
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    for a in soup.find_all("a"):
        a.decompose()
    return soup.get_text(" ", strip=True)

f_content['review_content'] = f_content['review_content'].apply(remove_html)
i_content['review_content'] = i_content['review_content'].apply(remove_html)

/tmp/ipython-input-552648710.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i_content['review_content'] = i_content['review_content'].apply(remove_html)


In [8]:
#Remove all regular expressions for a smooth encoding
def remove_poc(text):
  return re.sub(r"[^\w\s]", " ", text)

f_content['review_content'] = f_content['review_content'].apply(remove_poc)
i_content['review_content'] = i_content['review_content'].apply(remove_poc)

#Lower case the text
f_content["review_content"] = f_content["review_content"].str.lower()
i_content["review_content"] = i_content["review_content"].str.lower()

#Remove the content that becomes empty (like the row 998 in the fightclub data)
f_content = f_content[f_content["review_content"].str.strip() != ""]
i_content = i_content[i_content["review_content"].str.strip() != ""]

#Drop index
f_content = f_content.reset_index(drop=True)
i_content = i_content.reset_index(drop=True)

/tmp/ipython-input-3893709031.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i_content['review_content'] = i_content['review_content'].apply(remove_poc)
/tmp/ipython-input-3893709031.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i_content["review_content"] = i_content["review_content"].str.lower()


In [9]:
#Load Bert encoder and turn our contents into vector embeddings
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
f_emb = model.encode(f_content["review_content"].tolist())
i_emb = model.encode(i_content["review_content"].tolist())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
print(f_emb.shape)
print(i_emb.shape)

(995, 384)
(1000, 384)


#**2 .Recommendation system**

In [ ]:
def get_similar_reviews(df, cleaned_df, embeddings, source_id, k):
    """Find k most similar reviews using cosine similarity."""

    #Find the review index
    mask = cleaned_df["id"] == source_id
    if not mask.any():
        raise ValueError(f"Source ID {source_id} not found in dataset")

    source_idx = cleaned_df.index[cleaned_df["id"] == source_id][0]
    query_embedding = embeddings[source_idx].reshape(1, -1)

    #Calculate similarities
    similarities = cosine_similarity(embeddings, query_embedding).ravel()

    #Remove the current content
    similarities[source_idx] = -1

    #Get top k similar items
    top_indices = np.argsort(-similarities)[:k]
    top_ids = cleaned_df.iloc[top_indices]["id"].tolist()

    #Results
    results = cleaned_df.iloc[top_indices].copy()
    original_reviews = df.set_index("id").loc[top_ids, "review_content"]

    return pd.DataFrame({
        "cleaned_content": results["review_content"].values,
        "original_content": original_reviews.values,
        "id": top_ids,
        "similarity_score": similarities[top_indices].round(3)
    })

#**3 .Test**

In [ ]:
source_id = int(input('Enter current id '))
print("\nCurrent content :", fightclub[fightclub['id'] == source_id]['review_content'].values[0],'\n')
result_df = get_similar_reviews(fightclub,f_content, f_emb, source_id, 5)
result_df

Enter current id 6090599

Current content : Après l'énorme succès en 1995 de Seven et le plutôt réussi The Game en 1997, David Fincher était attendu au tournant avec son prochain film. Son nom: Fight Club. Dès l'introduction du DVD dans le lecteur, le ton est donné. Le traditionnel message d'avertissement contre la copie est parodié et est qualifié de perte de temps. Le film sera donc cynique, corrosif et il semble que rien ne sera épargné. Outre une mise en scène nerveuse et réussie, notamment dans les combats que se font les personnes à mains nues, Fincher critique la société de consommation actuelle, le culte de l'argent et du confort. En effet, le film est composé de scènes cultes. La présentation de l'appartement de Jack, à la manière d'un catalogue Ikea (d'ailleurs tout vient de ce magasin), ce dernier fortement critiqué puisqu'il fait partie de ces chaînes de magasins employant de nombreux moyens pour se montrer partout et faire rentabiliser ses caisses. Par défaut, l'homme est 

,cleaned_content,original_content,id,similarity_score
0,le 15 octobre 1999 david fincher a déposé une...,"<p>Le 15 octobre 1999, David Fincher a déposé ...",182277849,0.812
1,fight club sort à la fin 1999 sur les écrans ...,Fight Club sort à la fin 1999 sur les écrans. ...,20761,0.806
2,critique garantie avec spoilers fight club...,[Critique garantie avec spoilers] « Fight club...,45495675,0.798
3,a force de trop vendre ce film on finit par f...,"<p>A force de trop vendre ce film, on finit pa...",55931335,0.784
4,critique violente de la société de consommatio...,<p>Critique violente de la société de consomma...,85522023,0.780


In [ ]:
source_id = int(input('Enter current id '))
print("\nCurrent content :", interstellar[interstellar['id'] == source_id]['review_content'].values[0],'\n')
result_df = get_similar_reviews(interstellar, i_content, i_emb, source_id, 5)
result_df

Enter current id 25246858

Current content : Aïe Aïe Aïe, nous y voilà, Interstellar, le film dont on ne doit pas prononcer le nom, celui qui déchaîne les passions, film de la décennie pour certains, arnaque pour d'autres. Déjà moqué pour ces symboles et idées véhiculées, décrit comme Melo larmoyant, ou encore mascarade de Science Fiction. 2001 l'Odyssey de l'Espace doit figurer à chaque paragraphe, on doit bien sûr mentionner du Tarkovski, ou des récits avec une notoriété moins évidente comme Contact ou The Black Hole et enfin faire le lien avec Gravity évidemment.

Plutôt que d'aller puiser dans des centaines de références et d'influence diverses et variées, je considère Interstellar comme le film le plus personnel de Nolan, celui dans lequel il finit par assumer un message martelé depuis toujours. Réalisateur régulièrement critiqué pour sa froideur, sa prétention ou son ambition mal placée, il est pourtant à mon sens un grand romantique méconnu. L'amour est un sentiment qui transpir

,cleaned_content,original_content,id,similarity_score
0,par où commencer pour parler de ce film \n\n...,Par où commencer pour parler de ce film ? \n\n...,30188799,0.834
1,il faut croire qu avant d être un film de scie...,Il faut croire qu'avant d'être un film de scie...,29180546,0.810
2,en s inspirant librement aux films de science ...,En s'inspirant librement aux films de science-...,43065223,0.804
3,je viens de lire une critique qui déplore le m...,Je viens de lire une critique qui déplore le m...,45793453,0.795
4,bon ben voila j ai vu interstellar le film d...,"Bon ben voila, j'ai vu Interstellar, le film d...",41157121,0.790
